In [130]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.offline as pof
import scipy.stats as sps
sns.set()

plt.rc('font', size=30)
plt.rc('axes', titlesize=30)
plt.rc('axes', labelsize=30)
plt.rc('xtick', labelsize=30)
plt.rc('ytick', labelsize=30)
plt.rc('legend', fontsize=30)
plt.rc('figure', titlesize=30)

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import NMF
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.covariance import MinCovDet
import cvxpy as cvx
import yfinance as yf
from tqdm import tqdm

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from warnings import filterwarnings
filterwarnings('ignore')

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import  risk_models
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
from pypfopt import objective_functions

In [131]:
from rpy2.robjects.packages import importr
# import R's "base" package
base = importr('base')

# import R's "utils" package
utils = importr('utils')

# import rpy2's package module
import rpy2.robjects.packages as rpackages

# import R's utility package
utils = rpackages.importr('utils')

# select a mirror for R packages
utils.chooseCRANmirror(ind=1) # select the first mirror in the list

# R package names
packnames = ('NMF', 'nsprcomp', 'BiocManager')

# R vector of strings
from rpy2.robjects.vectors import StrVector

# Selectively install what needs to be install.
# We are fancy, just because we can.
names_to_install = [x for x in packnames if not rpackages.isinstalled(x)]
if len(names_to_install) > 0:
    utils.install_packages(StrVector(names_to_install))

nsprcomp = importr('nsprcomp')

# Allow conversion
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
pandas2ri.activate()

In [132]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import  risk_models
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
from pypfopt import objective_functions

In [133]:
data = pd.read_csv('yahoo_389c_5047r.csv')
prices = data.dropna(axis=1).set_index('Date')

In [134]:
returns = prices.pct_change(1).fillna(0)

In [135]:
train_size = 4000
returns_train = returns[:train_size]
returns_test = returns[train_size:]

In [136]:
mean = expected_returns.mean_historical_return(returns_train, returns_data=True)

S = risk_models.sample_cov(returns_train, returns_data=True) # for sample covariance matrix
S

,MMM,AOS,ABT,ABMD,ACN,ATVI,ADM,ADBE,AAP,AMD,...,WDC,WY,WHR,WMB,XEL,XLNX,YUM,ZBRA,ZBH,ZION
MMM,0.045598,0.033710,0.018203,0.031828,0.025569,0.028228,0.027594,0.035333,0.022744,0.040662,...,0.036435,0.038395,0.039544,0.041460,0.017168,0.033802,0.027173,0.028050,0.022595,0.045241
AOS,0.033710,0.112326,0.021339,0.050698,0.032417,0.039853,0.035405,0.047719,0.033044,0.065087,...,0.058775,0.052507,0.054534,0.055940,0.020336,0.044371,0.035173,0.042551,0.028891,0.060671
ABT,0.018203,0.021339,0.048871,0.028112,0.019129,0.021884,0.019485,0.024048,0.017128,0.028361,...,0.024382,0.023047,0.024419,0.031177,0.016637,0.022747,0.019336,0.019235,0.023375,0.026580
ABMD,0.031828,0.050698,0.028112,0.301869,0.040525,0.043321,0.032509,0.050653,0.030115,0.065234,...,0.063385,0.048965,0.056764,0.060066,0.023837,0.048190,0.029835,0.046750,0.032691,0.066206
ACN,0.025569,0.032417,0.019129,0.040525,0.089862,0.037319,0.027763,0.040438,0.024902,0.053149,...,0.042057,0.037993,0.038189,0.054779,0.018790,0.042138,0.023988,0.033327,0.023417,0.040363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XLNX,0.033802,0.044371,0.022747,0.048190,0.042138,0.055537,0.032855,0.065453,0.031001,0.107235,...,0.078174,0.047618,0.051845,0.062411,0.028998,0.132780,0.032278,0.047252,0.025024,0.050837
YUM,0.027173,0.035173,0.019336,0.029835,0.023988,0.029476,0.026650,0.036660,0.029332,0.043443,...,0.038556,0.037730,0.041427,0.037950,0.017427,0.032278,0.076235,0.031446,0.023972,0.046792
ZBRA,0.028050,0.042551,0.019235,0.046750,0.033327,0.038841,0.032511,0.044600,0.031097,0.065892,...,0.063233,0.043307,0.046219,0.063565,0.018421,0.047252,0.031446,0.107774,0.027100,0.048439
ZBH,0.022595,0.028891,0.023375,0.032691,0.023417,0.025548,0.024334,0.031423,0.023817,0.036597,...,0.032986,0.034469,0.035813,0.039405,0.015061,0.025024,0.023972,0.027100,0.073332,0.039087


In [137]:
ef = EfficientFrontier(mean, S)
weights = ef.max_sharpe()

In [138]:
weights

OrderedDict([('MMM', 0.0),
             ('AOS', 0.0),
             ('ABT', 0.0),
             ('ABMD', 0.0),
             ('ACN', 0.0),
             ('ATVI', 0.0),
             ('ADM', 0.0),
             ('ADBE', 0.0),
             ('AAP', 0.0),
             ('AMD', 0.0),
             ('AES', 0.0),
             ('AFL', 0.0),
             ('A', 0.0),
             ('APD', 0.0),
             ('AKAM', 0.0),
             ('ALK', 0.0),
             ('ALB', 0.0),
             ('ARE', 0.0),
             ('ALGN', 0.002232545466802),
             ('LNT', 0.0),
             ('ALL', 0.0),
             ('MO', 0.1369759463529149),
             ('AMZN', 0.037568793726003),
             ('AEE', 0.0),
             ('AEP', 0.0),
             ('AXP', 0.0),
             ('AIG', 0.0),
             ('AMT', 0.0),
             ('ABC', 0.0),
             ('AME', 0.0),
             ('AMGN', 0.0),
             ('APH', 0.0),
             ('ADI', 0.0),
             ('ANSS', 0.0),
             ('ANTM', 0.0),
      

In [191]:
def count_portfolio_return(weights, test):
    portfolio_return = 0
    for col in test.columns:
        one_company_return = weights[col] * (test[col] + 1).cumprod()
        portfolio_return += one_company_return
    return portfolio_return

def return_weights(df, R=2e-3):
    mean = expected_returns.mean_historical_return(df, returns_data=True)

    S = risk_models.sample_cov(df, returns_data=True)
    
    ef = EfficientFrontier(mean, S)
    weights = ef.max_sharpe()
    print(ef.portfolio_performance(verbose=True))
    return weights

def return_weights_PCA(df, R=2e-3):
    mean = expected_returns.mean_historical_return(df, returns_data=True)

    S = risk_models.sample_cov(df, returns_data=True)
    n_comp = 1
    pca = PCA(n_components=n_comp) 
    pca.fit(S)
    S = pca.get_covariance()
    
    ef = EfficientFrontier(mean, S)
    ef.add_objective(objective_functions.L2_reg, gamma=0.1)
    weights = ef.max_sharpe()
    print(ef.portfolio_performance(verbose=True))
    return weights

def plot_returns(names, returns, title, save=False):
    fig = go.Figure()
    
    for name, portfolio_return in zip(names, returns):
        fig.add_trace(go.Scatter(x=portfolio_return.index, y=portfolio_return, name=name))
            
    fig.update_layout(title=title, xaxis_title='date', 
                  yaxis_title='return')
    fig.show()
    if save:
        pof.plot(fig, filename=f'{title}.html', auto_open=False)

In [157]:
def sharpe_ratio(returns, N=255, rf=0.01):
    pd_r = pd.DataFrame(returns)
    pd_r = pd_r.pct_change()
    mean = pd_r.mean() * N - rf
    std_neg = pd_r.std() * np.sqrt(N)
    return (mean/std_neg)[pd_r.columns[0]]

def VAR_ratio(returns, quantile=0.05):
    return np.quantile(returns, quantile)

def sortino_ratio(returns, N=255,rf=0.01):
    pd_r = pd.DataFrame(returns)
    pd_r = pd_r.pct_change()
    mean = pd_r.mean() * N - rf
    std_neg = pd_r[pd_r<0].std() * np.sqrt(N)
    return (mean/std_neg)[pd_r.columns[0]]

In [158]:
def decrease_risk(mu, Sigma, R=2e-3):
    p = len(mu)
    w = cvx.Variable(p)
    obj = cvx.Minimize(1/2 * cvx.quad_form(w, Sigma))
    
    equal_constraints_1 = [mu.T @ w == R]
    equal_constraints_2 = [np.ones(p) @ w == 1]
    eyes = np.eye(p)
    nonequal_constraints = [eye @ w >= 0 for eye in eyes]
    constraints = equal_constraints_1 + equal_constraints_2 + nonequal_constraints
    
    problem = cvx.Problem(obj, constraints=constraints)
    result = problem.solve(verbose=False, solver="SCS")
    
    opt_weights = w.value
    return opt_weights

def count_portfolio_return_2(weights, test):
    portfolio_return = 0
    for w, col in zip(weights, test.columns):
        one_company_return = w * (test[col] + 1).cumprod()
        portfolio_return += one_company_return
    return portfolio_return

def return_weights_2(df, R=2e-3):
    mean = df[df.columns].mean()
    df_sorted = df[mean.sort_values(ascending=False).index]
    
    mu = mean.sort_values(ascending=False).to_numpy()
    Sigma = df_sorted[df_sorted.columns].cov()
    
    w = decrease_risk(mu, Sigma, R)
    return w

def get_mu(df):
    mean = df[df.columns].mean()    
    mu = mean.sort_values(ascending=False).to_numpy()
    return mu

def get_Sigma(df):
    Sigma = df[df.columns].cov()
    return Sigma

In [159]:
def NPCA_dim_red(df, n_comp = 2, window_size = 10):
    time_size = len(df.index)
    
    new_prices = list()

    for i in range(0, time_size, window_size):
        var = nsprcomp.nsprcomp(df[i : i + window_size].T, ncomp=n_comp, center = False, scale = False, nneg = True)
        new_prices.append(SVP(var[6].T))
    
    new_prices = np.vstack(new_prices)
    df_new_prices = pd.DataFrame(data=new_prices, index=np.arange(0, len(new_prices)), columns=df.columns)
    new_returns = df_new_prices.pct_change(1).fillna(0)
    new_returns = new_returns.drop(new_returns.index[0])
    
    return new_returns

def NMF_dim_red(df_prices, n_comp=1, window_size=5):
    new_prices = list()
    heights = list()

    for i in range(0, len(df_prices.index), window_size):
        model = NMF(n_components=2)
        W = model.fit_transform(df_prices[i : i + window_size].T)
        H = model.components_
        heights.append(H)
        new_prices.append(SVP(W.T))

    new_prices = np.vstack(new_prices)
    df_new_prices = pd.DataFrame(data=new_prices, index=np.arange(0, len(new_prices)), columns=df_prices.columns)
    new_returns = df_new_prices.apply(lambda x: x.pct_change(1).fillna(0), axis=0)
    return new_returns

def SVP(components):
    r = len(components)
    s = np.zeros(r)
    
    for i in range(r):
        s[i] = sum((components[i] - np.mean(components))**2) / (len(components[0]) - 1)
    
    c = np.zeros(r)
    
    for i in range(r):
        c[i] = s[i] / sum(s)
        
    ans = components.T @ c
    return ans

In [195]:
npca_returns = NPCA_dim_red(prices[:train_size], 2, 5)

In [196]:
npca_1_w = return_weights(npca_returns)

Expected annual return: 389.3%
Annual volatility: 40.0%
Sharpe Ratio: 9.68
(3.8929256033992896, 0.39992778347633473, 9.684062381798652)


In [197]:
npca_1_portfolio_return = count_portfolio_return(npca_1_w, returns_test)

In [198]:
sortino_ratio(npca_1_portfolio_return), npca_1_portfolio_return

(1.4896632405256496,
 Date
 2017-11-02    0.996506
 2017-11-03    1.000946
 2017-11-06    0.998799
 2017-11-07    1.003034
 2017-11-08    1.006585
                 ...   
 2021-12-23    2.648080
 2021-12-27    2.689960
 2021-12-28    2.686970
 2021-12-29    2.702720
 2021-12-30    2.691590
 Name: MMM, Length: 1047, dtype: float64)

In [199]:
npca_2_w = return_weights_2(npca_returns)
npca_2_portfolio_return = count_portfolio_return_2(npca_2_w, returns_test)
sortino_ratio(npca_2_portfolio_return), npca_2_portfolio_return

(0.897693682455824,
 Date
 2017-11-02    1.003787
 2017-11-03    1.004364
 2017-11-06    0.999585
 2017-11-07    0.994832
 2017-11-08    0.997639
                 ...   
 2021-12-23    1.850866
 2021-12-27    1.887644
 2021-12-28    1.886407
 2021-12-29    1.895195
 2021-12-30    1.889817
 Name: MMM, Length: 1047, dtype: float64)

In [164]:
basic_1_w = return_weights(returns_train)

Expected annual return: 28.4%
Annual volatility: 18.2%
Sharpe Ratio: 1.45
(0.2840909559380936, 0.1816315990088499, 1.4539923525378746)


In [165]:
basic_1_portfolio_return = count_portfolio_return(basic_1_w, returns_test)
sortino_ratio(basic_1_portfolio_return)

1.4895998724330484

In [168]:
basic_2_w = return_weights_2(returns_train)
basic_2_portfolio_return = count_portfolio_return_2(basic_2_w, returns_test)
sortino_ratio(basic_2_portfolio_return)

0.21424247343528183

In [200]:
portfolio_returns = [npca_1_portfolio_return, npca_2_portfolio_return, basic_1_portfolio_return, basic_2_portfolio_return]
names = ['NPCA_1', 'NPCA_2', 'without opt 1', 'without opt 2']
plot_returns(names, portfolio_returns, 'portfolio returns plot')

In [171]:
nmf_returns = NMF_dim_red(prices[:train_size], 2, 5)
nmf_1_w = return_weights(nmf_returns)
nmf_1_portfolio_return = count_portfolio_return(nmf_1_w, returns_test)
sortino_ratio(nmf_1_portfolio_return), nmf_1_portfolio_return

Expected annual return: 369.6%
Annual volatility: 60.7%
Sharpe Ratio: 6.06
(3.6957386249131385, 0.6069864568929672, 6.055717690520563)


(1.4008108769027177,
 Date
 2017-11-02    0.999786
 2017-11-03    1.011733
 2017-11-06    1.014733
 2017-11-07    1.018870
 2017-11-08    1.023157
                 ...   
 2021-12-23    2.865741
 2021-12-27    2.912243
 2021-12-28    2.905228
 2021-12-29    2.916186
 2021-12-30    2.899521
 Name: MMM, Length: 1047, dtype: float64)

In [173]:
nmf_2_w = return_weights_2(nmf_returns)
nmf_2_portfolio_return = count_portfolio_return_2(nmf_2_w, returns_test)
sortino_ratio(nmf_2_portfolio_return), nmf_2_portfolio_return

(0.8778661026434104,
 Date
 2017-11-02    1.004650
 2017-11-03    1.005515
 2017-11-06    1.009699
 2017-11-07    1.011734
 2017-11-08    1.016290
                 ...   
 2021-12-23    1.816251
 2021-12-27    1.841859
 2021-12-28    1.844177
 2021-12-29    1.848640
 2021-12-30    1.846289
 Name: MMM, Length: 1047, dtype: float64)

In [201]:
portfolio_returns = [npca_1_portfolio_return, npca_2_portfolio_return, 
                     basic_1_portfolio_return, basic_2_portfolio_return,
                     nmf_1_portfolio_return, nmf_2_portfolio_return]
names = ['NPCA_1', 'NPCA_2', 'without opt 1', 'without opt 2', 'NMF_1', 'NMF_2']
plot_returns(names, portfolio_returns, 'portfolio returns plot')